In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import datetime as dt
import os
from glob import glob

## Data Preprocessing

### Data Loading

all data in one big DataFrame

In [3]:
curr_path = os.getcwd()
root_path = curr_path + "/data/OHLCV.parquet"
all_coins = sorted(glob(os.path.join(root_path, "coin_id=*")))

df_all_coins = []

for coin_path in all_coins:
    coin_id = os.path.basename(coin_path).split("=")[1]  #  BTCUSDT etc
    
    year_dirs = sorted(glob(os.path.join(coin_path, "year=*")))
    
    for year_path in year_dirs:
        files = sorted(glob(os.path.join(year_path, "*.parquet")))
        for f in files:
            df = pd.read_parquet(f)
            df["coin_id"] = coin_id  # type of coin
            df_all_coins.append(df)

# One Big DataFrame
df_all = pd.concat(df_all_coins, ignore_index=True)
# Convert timestamp to datetime
df_all['datetime'] = pd.to_datetime(df_all['timestamp'], unit='ms')
# sort by coin_id and timestamp
df_all = df_all.sort_values(['coin_id', 'datetime']).set_index('datetime')

In [4]:
print(type(df_all))
print(df_all.shape)
print(df_all.columns)
print(df_all.dtypes)
print(df_all.head())

<class 'pandas.core.frame.DataFrame'>
(11349947, 12)
Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
       'taker_buy_quote_asset_volume', 'coin_id'],
      dtype='object')
timestamp                       datetime64[ns]
open                                   float64
high                                   float64
low                                    float64
close                                  float64
volume                                 float64
close_time                              object
quote_asset_volume                     float64
number_of_trades                         int64
taker_buy_base_asset_volume            float64
taker_buy_quote_asset_volume           float64
coin_id                                 object
dtype: object
                              timestamp     open     high      low    close  \
datetime                                                     

In [5]:
all_coins = sorted(glob(os.path.join(root_path, "coin_id=*")))
sorted_coin_ids = df_all['coin_id'].unique()
print(sorted_coin_ids)

['BNBUSDT' 'BTCUSDT' 'ETHUSDT' 'SOLUSDT' 'XRPUSDT']


#### Data Inspection

In [6]:
# Missing Value Inspection
df_all.isnull().sum()

timestamp                       0
open                            0
high                            0
low                             0
close                           0
volume                          0
close_time                      0
quote_asset_volume              0
number_of_trades                0
taker_buy_base_asset_volume     0
taker_buy_quote_asset_volume    0
coin_id                         0
dtype: int64

In [7]:
# Check for duplicate rows
df_all.duplicated().sum()

np.int64(5)

In [10]:
# Show all duplicated rows (excluding the first occurrence)
duplicates = df_all[df_all.duplicated()]
print(duplicates)

                              timestamp      open        high         low  \
datetime                                                                    
2025-04-18 03:00:00 2025-04-18 03:00:00    589.79    590.4500    589.7700   
2025-04-18 03:00:00 2025-04-18 03:00:00  84888.00  84897.0600  84881.6700   
2025-04-18 03:00:00 2025-04-18 03:00:00   1584.57   1585.0200   1584.2100   
2025-04-18 03:00:00 2025-04-18 03:00:00    135.31    135.3600    135.2000   
2025-04-18 03:00:00 2025-04-18 03:00:00      2.06      2.0609      2.0593   

                          close       volume                  close_time  \
datetime                                                                   
2025-04-18 03:00:00    590.4400    327.21200  2025-04-18 03:00:59.999000   
2025-04-18 03:00:00  84881.6800      4.97719  2025-04-18 03:00:59.999000   
2025-04-18 03:00:00   1584.7000    133.30930  2025-04-18 03:00:59.999000   
2025-04-18 03:00:00    135.3400   1341.06500  2025-04-18 03:00:59.999000   
2025

In [12]:
duplicates_subset = df_all[df_all.duplicated(subset=['timestamp', 'coin_id'])]
print(duplicates_subset)

                              timestamp      open        high         low  \
datetime                                                                    
2025-04-18 03:00:00 2025-04-18 03:00:00    589.79    590.4500    589.7700   
2025-04-18 03:00:00 2025-04-18 03:00:00  84888.00  84897.0600  84881.6700   
2025-04-18 03:00:00 2025-04-18 03:00:00   1584.57   1585.0200   1584.2100   
2025-04-18 03:00:00 2025-04-18 03:00:00    135.31    135.3600    135.2000   
2025-04-18 03:00:00 2025-04-18 03:00:00      2.06      2.0609      2.0593   

                          close       volume                  close_time  \
datetime                                                                   
2025-04-18 03:00:00    590.4400    327.21200  2025-04-18 03:00:59.999000   
2025-04-18 03:00:00  84881.6800      4.97719  2025-04-18 03:00:59.999000   
2025-04-18 03:00:00   1584.7000    133.30930  2025-04-18 03:00:59.999000   
2025-04-18 03:00:00    135.3400   1341.06500  2025-04-18 03:00:59.999000   
2025

In [13]:
# Drop duplicates based on 'timestamp' and 'coin_id', keeping the first occurrence
df_all = df_all.drop_duplicates(subset=['timestamp', 'coin_id'], keep='first')

In [14]:
print("Remaining duplicates:", df_all.duplicated(subset=['timestamp', 'coin_id']).sum())

Remaining duplicates: 0


In [8]:
# Look for negative or zero prices (should not happen)
print((df_all[['open', 'high', 'low', 'close']] <= 0).sum())

open     0
high     0
low      0
close    0
dtype: int64


In [ ]:
# Find missing 1-minute timestamps for each coin
full_range = pd.date_range(
    start=df_all['timestamp'].min(),
    end=df_all['timestamp'].max(),
    freq='1min' 
)

missing_data = dict()
print(f"Full count of timestamps: {len(full_range)}.")

coin_types = df_all['coin_id'].unique()

for coin in coin_types:
    coin_df = df_all[df_all['coin_id'] == coin]
    print(f"Coin {coin} has {len(coin_df)} timestamps.", end=' ')
    
    missing_timestamps = full_range[~full_range.isin(coin_df['timestamp'])]
    missing_data[coin] = missing_timestamps
    
    print(f"Coin {coin} is missing {len(missing_timestamps)} timestamps.")

Full count of timestamps: 2271061.
Coin BNBUSDT has 2269988 timestamps. Coin BNBUSDT is missing 1073 timestamps.
Coin BTCUSDT has 2269988 timestamps. Coin BTCUSDT is missing 1073 timestamps.
Coin ETHUSDT has 2269988 timestamps. Coin ETHUSDT is missing 1073 timestamps.
Coin SOLUSDT has 2269989 timestamps. Coin SOLUSDT is missing 1072 timestamps.
Coin XRPUSDT has 2269989 timestamps. Coin XRPUSDT is missing 1072 timestamps.


#### Forward-fill

In [16]:
# Step 1: Build full timestamp index
full_range = pd.date_range(
    start=df_all['timestamp'].min(),
    end=df_all['timestamp'].max(),
    freq='1min'
)

# Step 2: Reindex and forward-fill per coin
df_filled_list = []

for coin in df_all['coin_id'].unique():
    coin_df = df_all[df_all['coin_id'] == coin].copy()
    coin_df = coin_df.set_index('timestamp')
    
    # Reindex to full timestamp grid
    coin_df = coin_df.reindex(full_range)
    
    # Restore coin_id
    coin_df['coin_id'] = coin

    # Forward-fill all columns except coin_id
    coin_df.update(coin_df.ffill())

    df_filled_list.append(coin_df)

# Step 3: Combine all coins back together
df_filled = pd.concat(df_filled_list).reset_index().rename(columns={'index': 'timestamp'})

In [17]:
# Sort for cleanliness
df_filled = df_filled.sort_values(['coin_id', 'timestamp'])

# Optional: Check again whether all coins now have full timestamps
print(df_filled.groupby('coin_id')['timestamp'].nunique())  # should match len(full_range)

coin_id
BNBUSDT    2271061
BTCUSDT    2271061
ETHUSDT    2271061
SOLUSDT    2271061
XRPUSDT    2271061
Name: timestamp, dtype: int64


#### 5-minute resampling

In [18]:
# Step 1: Ensure timestamp is in datetime format and set as index
df_filled['timestamp'] = pd.to_datetime(df_filled['timestamp'])
df_filled = df_filled.sort_values(['coin_id', 'timestamp'])
df_filled = df_filled.set_index('timestamp')

# Step 2: Group by coin and resample to 5-minute intervals
df_5min = (
    df_filled
    .groupby('coin_id')
    .resample('5T')  # 5-minute bins
    .agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'quote_asset_volume': 'sum',
        'number_of_trades': 'sum',
        'taker_buy_base_asset_volume': 'sum',
        'taker_buy_quote_asset_volume': 'sum'
    })
    .dropna(subset=['open'])  # remove empty bins
    .reset_index()
)

/var/folders/bj/t2psl0l57gz376bkt4cjp8l00000gn/T/ipykernel_28597/633000532.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample('5T')  # 5-minute bins


In [19]:
print(df_5min.groupby('coin_id')['timestamp'].nunique())  #  1/5 ≈ 454212 
print(df_5min.head())

coin_id
BNBUSDT    454213
BTCUSDT    454213
ETHUSDT    454213
SOLUSDT    454213
XRPUSDT    454213
Name: timestamp, dtype: int64
   coin_id           timestamp     open     high      low    close     volume  \
0  BNBUSDT 2021-01-01 00:00:00  37.3596  37.4221  37.2502  37.2639   5695.351   
1  BNBUSDT 2021-01-01 00:05:00  37.2596  37.2850  37.0743  37.1082  21319.204   
2  BNBUSDT 2021-01-01 00:10:00  37.1082  37.1546  36.9706  37.0022   8278.101   
3  BNBUSDT 2021-01-01 00:15:00  37.0134  37.1332  36.9636  37.1200   5000.406   
4  BNBUSDT 2021-01-01 00:20:00  37.1186  37.1302  37.0341  37.0935   6107.471   

   quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
0       212739.076283             799.0                     2928.143   
1       791937.021860            1762.0                    12079.071   
2       306915.404129            1319.0                     3180.308   
3       185188.792115             940.0                     2133.491   
4       226489.159293    

#### Save cleaned data

In [20]:
# Save cleaned 1-minute data
df_filled.to_parquet("data/cleaned_1min.parquet", index=False)

# Save resampled 5-minute data
df_5min.to_parquet("data/resampled_5min.parquet", index=False)